In [1]:
import hnswlib
import numpy as np
from libs.nodes import *

# Approx Graph w/ Inferred Nodes via KNN

In [10]:
def nodes_to_matrix(nodes):
    dim = max([max(x) for x in nodes]) # attributes are 1-indexed
    node_amnt = len(nodes)

    # Convert to sparse matrix
    data = np.zeros((node_amnt, dim))
    for i in range(node_amnt):
        for x in nodes[i]:
            data[i][x-1] = 1
    
    return data


def build_index(data, ef_const, M):
    node_amnt = len(data)
    dim = len(data[0])

    # using l2 distance
    index = hnswlib.Index(space='l2', dim=dim)
    # index = hnswlib.BFIndex(space='l2', dim=dim)
    index.init_index(max_elements=node_amnt, ef_construction=ef_const, M=M)
    # index.init_index(max_elements=node_amnt)
    index.add_items(data)
    return index


def search(index, data, k, ef):
    k += 1 # first result is always the node itself
    index.set_ef(ef)
    
    labels, _ = index.knn_query(data, k=k)
    ids = [x[0] for x in labels]
    labels = [x[1:].tolist() for x in labels]
    results = list(zip(ids, labels))
    return results


def knn(path):
    nodes = parse_nodes(path)
    matrix = nodes_to_matrix(nodes)
    ef = 400
    index = build_index(matrix, ef_const=ef, M=50)
    results = search(index, matrix, k=100, ef=ef)
    # mapping back to nodes
    return [(nodes[root], [nodes[x] for x in nn]) for root, nn in results]

In [11]:
knn_pre_inf = knn("../../data/int_complete/setA/11935_out.nodes")
knn_in_time = knn("../../data/int_complete/setA/212_in.nodes")

In [12]:
knn_in_time[:10]

[((1,2,3,5,11,42,68,84,103,228,238,264,295,310,327,329,353,468,496,563,570,673,679,680,681,682,685),
  [(1,2,3,5,11,42,68,103,228,264,295,310,327,329,353,468,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,228,238,264,295,310,327,329,353,422,468,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,228,238,264,295,310,327,329,353,468,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,85,103,228,238,264,295,310,327,329,353,422,468,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,110,228,238,264,295,310,327,329,353,422,468,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,228,238,264,295,310,327,329,346,353,422,468,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,228,238,264,295,310,327,329,353,422,468,483,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,228,238,244,264,295,310,327,329,353,422,468,496,563,570,673,679,680,681,682,685),
   (1,2,3,5,11,42,68,103,228,238,264,295,310,327,329,333,353,422,468,496

## In-time results produce inferred nodes from current nodes

In [14]:
def expand_candidates(pre_inf: list[node, list[node]]):
    def infer(root: node, candidates: list[node]): 
        inf = set(candidates)
        for candidate in candidates:
            inf.add(candidate & root)
        return inf

    return [(root, infer(root, candidates)) for root, candidates in pre_inf]

nodes = [
    node({1,2,3,4}),
    node({2,3,4,5}),
    node({4,5,8,9})
]

test_knn = [
    (0, [1, 2]),
    (1, [0]),
    (2, [0, 1])
]

test_data = [(nodes[root], [nodes[x] for x in nn]) for root, nn in test_knn]
expand_candidates(test_data)

[((1,2,3,4), {(4), (4,5,8,9), (2,3,4), (2,3,4,5)}),
 ((2,3,4,5), {(2,3,4), (1,2,3,4)}),
 ((4,5,8,9), {(4), (4,5), (1,2,3,4), (2,3,4,5)})]

In [16]:
knn_in_time = expand_candidates(knn_in_time)

## Filtering for distance $\le 5$

In [17]:
def filter(results: list[tuple[node, list[node]]], d=5):
    close = lambda root, list: [x for x in list if x & root >= d]
    return [(root, close(root, candidates)) for root, candidates in results]


test_data = [
    [node({1,2,3,4}), [node({1,2,3}), node({1,2,3,4,5})]],
    [node({2,3,4,5}), [node({1,2,9}), node({1,2,4,5})]],
    [node({4,5,8,9}), [node({1,2,3,4,5}), node({1,2,3,4,5,6}), node({4, 5}), node({4})]]
]

filter(test_data, d=2)

[((1,2,3,4), [(1,2,3), (1,2,3,4,5)]),
 ((2,3,4,5), [(1,2,4,5)]),
 ((4,5,8,9), [(1,2,3,4,5), (1,2,3,4,5,6), (4,5)])]

In [18]:
d = 5
candidates_pre_inf = filter(knn_pre_inf, d)
candidates_in_time = filter(knn_in_time, d)